## Clasificacion de imagenes de retinas

Este documento detalla las lineas de codigos utilizadas para armar una base datos de intesidades de pixeles de interes y su posterior utilizacion para la elaborar redes neuronales que distingue una capa de la otra

In [1]:
import cv2
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
import random
import numpy as np


R_1=cv2.imread(r'D:\Imagenes de Retina\Im_retina.tif')
R_2=cv2.imread(r'D:\Imagenes de Retina\Im_retina2.tif')
R_3=cv2.imread(r'D:\Imagenes de Retina\Im_retina3.tif')
R_4=cv2.imread(r'D:\Imagenes de Retina\Im_retina4.tif')

### las imagenes las rote en photoshop para poder despues elegir los rectangulos de la roi de manera mas sencilla


## ROI

En este paso se seleccionaron las roi de las capas a analizar. Previamente, las imagenes fueron rotadas en photoshop para ser selecionadas de manera mas sencilla ya que la herramienta de CV2 solo permite seleccionar rectangulos y no poligonos inrregulares.

In [ ]:
showCrosshair = False
fromCenter = False
roi1_oscuro = cv2.selectROI( R_1, fromCenter, showCrosshair)
roi2_oscuro = cv2.selectROI( R_2, fromCenter, showCrosshair)
roi3_oscuro = cv2.selectROI( R_3, fromCenter, showCrosshair)
roi4_oscuro = cv2.selectROI( R_4, fromCenter, showCrosshair)
roi1_clara = cv2.selectROI( R_1, fromCenter, showCrosshair)
roi2_clara = cv2.selectROI( R_2, fromCenter, showCrosshair)
roi3_clara = cv2.selectROI( R_3, fromCenter, showCrosshair)
roi4_clara = cv2.selectROI( R_4, fromCenter, showCrosshair)
cv2.waitKey()



In [ ]:

r1_o=[117,219,1084,97]
r2_o=[64,411,1011,217]
r3_o=[2,338,1022,203]
r4_o=[734,3,155,675]

r1_c=[99,373,1162,54]
r2_c=[433,218,725,111]
r3_c=[3,618,878,77]
r4_c=[574,2,117,723]

In [ ]:
imagen1_roi_O=R_1[r1_o[1]:(r1_o[1]+r1_o[3]),r1_o[0]:(r1_o[0]+r1_o[2])]
imagen2_roi_O=R_2[r2_o[1]:(r2_o[1]+r2_o[3]),r2_o[0]:(r2_o[0]+r2_o[2])]
imagen3_roi_O=R_3[r3_o[1]:(r3_o[1]+r3_o[3]),r3_o[0]:(r3_o[0]+r3_o[2])]
imagen4_roi_O=R_4[r4_o[1]:(r4_o[1]+r4_o[3]),r4_o[0]:(r4_o[0]+r4_o[2])]

imagen1_roi_C=R_1[r1_c[1]:(r1_c[1]+r1_c[3]),r1_c[0]:(r1_c[0]+r1_c[2])]
imagen2_roi_C=R_2[r2_c[1]:(r2_c[1]+r2_c[3]),r2_c[0]:(r2_c[0]+r2_c[2])]
imagen3_roi_C=R_3[r3_c[1]:(r3_c[1]+r3_c[3]),r3_c[0]:(r3_c[0]+r3_c[2])]
imagen4_roi_C=R_4[r4_c[1]:(r4_c[1]+r4_c[3]),r4_c[0]:(r4_c[0]+r4_c[2])]

In [ ]:
i1_r_O=imagen1_roi_O[:,:,1]
i2_r_O=imagen2_roi_O[:,:,1]
i3_r_O=imagen3_roi_O[:,:,1]
i4_r_O=imagen4_roi_O[:,:,1]

i1_r_C=imagen1_roi_C[:,:,1]
i2_r_C=imagen2_roi_C[:,:,1]
i3_r_C=imagen3_roi_C[:,:,1]
i4_r_C=imagen4_roi_C[:,:,1]

In [ ]:
imagenes_oscuras=[i1_r_O,i2_r_O,i3_r_O,i4_r_O]
imagenes_clara=[i1_r_C,i2_r_C,i3_r_C,i4_r_C]

## Selecciones de pixeles dentro de las rois

Una vez recortadas las imagenes en las regiones de interes, elaboramos los siguientes algoritmos que van seleccionando 6000 pixeles y sus vecinos de cada imagen. Ademas se le agrego un sexto elemento al vector de cada quintupla de pixeles que indica el grupo de celulas a quien pertenecen.

In [ ]:
datos_o=None

for r in range (4):
    for p in range (6000):
        fil=random.randint(10,imagenes_oscuras[r].shape[0]-10)
        col=random.randint(10,imagenes_oscuras[r].shape[1]-10)        
        pixeles_o = [imagenes_oscuras[r][fil, col], imagenes_oscuras[r][fil+5, col], imagenes_oscuras[r][fil, col+5], imagenes_oscuras[r][fil-5,col],imagenes_oscuras[r][fil,col-5],0]
        print('voy por los pixels ',r,fil,col)
        if datos_o is None:
              datos_o=pixeles_o
        else:
              datos_o= np.vstack((datos_o,pixeles_o))

In [ ]:
datos_c=None

for r in range (4):
    for p in range (6000):
        fil=random.randint(10,imagenes_clara[r].shape[0]-10)
        col=random.randint(10,imagenes_clara[r].shape[1]-10)        
        pixelesc = [imagenes_clara[r][fil, col], imagenes_clara[r][fil+5, col], imagenes_clara[r][fil, col+5], imagenes_clara[r][fil-5,col],imagenes_clara[r][fil,col-5],1]
        print('voy por los pixels ',r,fil,col)
        if datos_c is None:
              datos_c=pixelesc
        else:
              datos_c= np.vstack((datos_c,pixelesc))

## Elaboracion de base de datos

Ya seleccionadas la intesidad de los pixeles de cada grupo, armamos una base de datos y posteriormente las desoordenamos al azar ya que los datos estaban distruido de manera uniforme. Los primeros 24000 pertenecian a la categoria 0 y los segundo a la 1.

In [ ]:
base_datos=np.vstack((datos_o,datos_c))
base_datos.shape

In [ ]:
np.take(base_datos,np.random.permutation(base_datos.shape[0]),axis=0,out=base_datos)

## Separamos los datos en entranamiento y testeo

Como los datos estan distribuidos de manera aleatoria en la base podemos selecionar los primeros 40000 para entrenamiento y los otros 8000 para testeo. Luego verificamos si entan distribuido de manera uniforme

In [ ]:

X_train=None
Y_train=None
for p in range (40000):
    xt=base_datos[p][0:5]
    yt=base_datos[p][5]
    if X_train is None:
              X_train=xt
              Y_train=  yt
    else:
        Y_train= np.vstack((Y_train,yt))
        X_train= np.vstack((X_train,xt))
            

In [ ]:
X_test=None
Y_test=None
for p in range (40001, 48000):
    xte=base_datos[p][0:5]
    yte=base_datos[p][5]
    if X_test is None:
              X_test=xte
              Y_test=yte
    else:
        Y_test= np.vstack((Y_test,yte))
        X_test= np.vstack((X_test,xte))

In [ ]:

print(Y_train.sum()/len(Y_train))
print(Y_test.sum()/len(Y_test))

## Elaboracion de las redes neuronales

 Armamos dos redes neuronales distintas para poder comparar sus rendimientos.

In [ ]:
Y_train = keras.utils.to_categorical(Y_train)

In [ ]:
Y_test = keras.utils.to_categorical(Y_test)

In [ ]:
pixeles_entrada1 = layers.Input(shape = (5,))

In [ ]:
#ARMADO DE LA RED NEURONAL 1
Capa11 = layers.Dense(units = 42, activation = 'relu')(pixeles_entrada1)
Capa21 = layers.Dense(units = 200, activation = 'relu')(Capa11)
Salida1 = layers.Dense(units = 2, activation = 'softmax')(Capa21)
Red_Neuronal1 = keras.Model(inputs = pixeles_entrada1, outputs = Salida1)

In [ ]:
Red_Neuronal1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
Historia1 = Red_Neuronal1.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 32, validation_data = (X_test, Y_test))

In [ ]:
Capa12 = layers.Dense(units = 70, activation = 'sigmoid')(pixeles_entrada1)
Capa22 = layers.Dense(units = 221, activation = 'relu')(Capa12)
Capa32 = layers.Dense(units = 100, activation = 'sigmoid')(Capa22)
Salida2 = layers.Dense(units = 2, activation = 'softmax')(Capa32)
Red_Neuronal2 = keras.Model(inputs = pixeles_entrada1, outputs = Salida2)


In [ ]:
Red_Neuronal2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
Historia2 = Red_Neuronal2.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 64, validation_data = (X_test, Y_test))

In [ ]:
Capa13 = layers.Dense(units = 70, activation = 'sigmoid')(pixeles_entrada1)
Capa23 = layers.Dense(units = 221, activation = 'relu')(Capa13)
Capa33 = layers.Dense(units = 100, activation = 'sigmoid')(Capa23)
Capa43 = layers.Dense(units = 300, activation = 'relu')(Capa33)
Capa53 = layers.Dense(units = 40, activation = 'relu')(Capa43)
Salida3 = layers.Dense(units = 2, activation = 'softmax')(Capa53)
Red_Neuronal3 = keras.Model(inputs = pixeles_entrada1, outputs = Salida3)


In [ ]:
Red_Neuronal3.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
Historia3 = Red_Neuronal3.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 64, validation_data = (X_test, Y_test))

## Corroboramos el rendimiento de cada una


In [ ]:
plt.plot(Historia1.history['accuracy'], label = 'Entrenamiento de la red 1')
plt.plot(Historia1.history['val_accuracy'], label = 'Prueba de la red 1')
plt.plot(Historia2.history['accuracy'], label = 'Entrenamiento de la red 2')
plt.plot(Historia2.history['val_accuracy'], label = 'Prueba de la red 2')
plt.plot(Historia3.history['accuracy'], label = 'Entrenamiento de la red 3')
plt.plot(Historia3.history['val_accuracy'], label = 'Prueba de la red 3')
plt.grid()
plt.xlabel('Cantidad de Épocas')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Puesta a prueba - Imagen conocida

In [ ]:
step = 10
if step%2 != 0:
    step = step + 1

matriz_colores = np.zeros(imagen.shape)
for i in range(100, imagen.shape[0] - 50, step):
    for j in range(20, imagen.shape[1] - 200, step):
        vector = [imagen[i-5, j], imagen[i, j-5], ..]
        pred = Red_Neuronal2.predict(vector)
        
        if np.argmax(pred) == 0:
            matriz_colores[0, i-int(step/2): i+int(step/2)+1, j-int(step/2):j+int(step/2)+1] = 1
        elif np.argmax(pred) == :
            matriz_colores[1, i-int(step/2): i+int(step/2)+1, j-int(step/2):j+int(step/2)+1] = 1
            
plt.figure()
plt.title("Immagen orig")
plt.imshow(imagen)
plt.figure()
plt.title("Predicciones")
plt.imshow(matriz_colores)

# Puesta a prueba - Imagen desconocida

In [ ]:
#cargar R_5

In [ ]:
step = 10
if step%2 != 0:
    step = step + 1

matriz_colores = np.zeros(imagen.shape)
for i in range(100, imagen.shape[0] - 50, step):
    for j in range(20, imagen.shape[1] - 200, step):
        vector = [imagen[i-5, j], imagen[i, j-5], ..]
        pred = Red_Neuronal2.predict(vector)
        
        if np.argmax(pred) == 0:
            matriz_colores[0, i-int(step/2): i+int(step/2)+1, j-int(step/2):j+int(step/2)+1] = 1
        elif np.argmax(pred) == :
            matriz_colores[1, i-int(step/2): i+int(step/2)+1, j-int(step/2):j+int(step/2)+1] = 1
            
plt.figure()
plt.title("Immagen orig")
plt.imshow(imagen)
plt.figure()
plt.title("Predicciones")
plt.imshow(matriz_colores)